In [ ]:
project = "" # @param {type:"string"}
location = "" # @param {type:"string"}

# Example 1: Query a table with SQL and magic commands

Colab Notebooks in BigQuery supports execution of queries and generates results in a dataframe.
Below code queries the BQ table created by Spark in previous step, and provides the results in a Pandas Dataframe `revenue_report_df`

In [ ]:
# sql_engine: bigquery
# output_variable: revenue_report_df
# start _sql
_sql = """
SELECT * FROM `datalake_demo.revenue_report`
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
revenue_report_df = _bqsqlcell.run(_sql)
revenue_report_df

In [ ]:
# This is a Code cell, it has context so the Dataframe created by previous SQL
# cell is available here too.
revenue_report_df

In [ ]:
# sql_engine: bigquery
# output_variable: customers_df
# start _sql
_sql = """
-- BigLake External tables created for the migrated data are also accesible in BigQuery.
SELECT * FROM `datalake_demo.customers` LIMIT 10;
""" # end _sql
from google.colab.sql import bigquery as _bqsqlcell
customers_df = _bqsqlcell.run(_sql)
customers_df

# Example 2: Query a table with BigQuery DataFrames

With BigQuery DataFrames, you can use many familiar Pandas methods, but the processing happens in BigQuery rather than the runtime, allowing you to work with larger DataFrames that would otherwise not fit in the runtime memory.
Learn more here: https://cloud.google.com/python/docs/reference/bigframes/latest


In [ ]:

# This cell uses BigFrames to query the table generated by Spark previously.
import bigframes.pandas as bf

df = bf.read_gbq(f"{project}.datalake_demo.revenue_report")
df.head(20)

# Example 3: Query a table with remote Spark session

The code below shows how both BigLake External tables and BigQuery tables can be consumed from PySpark

In [ ]:
from google.cloud.dataproc_spark_connect import DataprocSparkSession

## Create a Spark session with default configuration:
spark = DataprocSparkSession.builder.getOrCreate()

In [ ]:
# The code below shows how BigLake External tables can be consumed from PySpark

# Load data from BigQuery.
table = spark.read.format('bigquery') \
  .option('table', f"{project}.datalake_demo.customers") \
  .load()
table.createOrReplaceTempView('customersTempView')

# Explore the data and schema.
table_data = spark.sql(
    'SELECT * FROM customersTempView')
table_data.show()
table_data.printSchema()

In [ ]:
# The code below shows how both BigQuery tables can be consumed from PySpark

# Load data from BigQuery.
table = spark.read.format('bigquery') \
  .option('table', f"{project}.datalake_demo.revenue_report") \
  .load()
table.createOrReplaceTempView('revenueReportTempView')

# Explore the data and schema.
table_data = spark.sql(
    'SELECT * FROM revenueReportTempView')
table_data.show()
table_data.printSchema()